# 優先会員ミドルウェアを活用したホテル予約

このノートブックでは、Microsoft Agent Framework を使用した**関数ベースのミドルウェア**を紹介します。条件付きワークフローの例を基に、優先会員に特別な特典を与えるミドルウェア層を追加します。

## 学べること:
1. **関数ベースのミドルウェア**: 関数の結果を傍受して変更する方法
2. **コンテキストアクセス**: 実行後に `context.result` を読み書きする方法
3. **ビジネスロジックの実装**: 優先会員特典の適用
4. **結果の上書き**: ユーザーのステータスに基づいて関数の結果を変更する方法
5. **同じワークフローで異なる結果**: ミドルウェアによる動作の変化

## ミドルウェアを活用したワークフローのアーキテクチャ:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## 条件付きワークフローとの主な違い:

**ミドルウェアなし** (14-conditional-workflow.ipynb):
- パリに空室なし → alternative_agent にルートされる

**ミドルウェアあり** (このノートブック):
- 通常ユーザー + パリ → 空室なし → alternative_agent にルートされる
- 優先会員 + パリ → 🌟 ミドルウェアが上書き！ → 空室あり → booking_agent にルートされる

## 前提条件:
- Microsoft Agent Framework がインストールされていること
- 条件付きワークフローの理解 (14-conditional-workflow.ipynb を参照)
- GitHub トークンまたは OpenAI API キー
- ミドルウェアパターンの基本的な理解


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ステップ 1: 構造化出力のためのPydanticモデルを定義する

これらのモデルは、エージェントが返す**スキーマ**を定義します。ミドルウェアが可用性結果を変更した際に追跡するための`priority_override`フィールドを追加しました。


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ステップ2: 優先メンバーのデータベースを定義する

このデモでは、優先メンバーシップのデータベースをシミュレーションします。本番環境では、実際のデータベースやAPIをクエリすることになります。

**優先メンバー:**
- `alice@example.com` - VIPメンバー
- `bob@example.com` - プレミアムメンバー  
- `priority_user` - テストアカウント


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## ステップ3: ホテル予約ツールを作成する

条件付きワークフローと同じですが、今回はミドルウェアによって処理が割り込まれます！


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ステップ 4: 🌟 優先チェックミドルウェアを作成 (重要な機能！)

これがこのノートブックの**核心的な機能**です。このミドルウェアは以下を行います：

1. ホテル予約関数の呼び出しを**傍受**します
2. `next(context)`を呼び出して関数を通常通り**実行**します
3. `context.result`内の結果を**確認**します
4. ユーザーが優先顧客で部屋が空いていない場合、結果を**上書き**します
5. 修正された結果をエージェントに**返却**します

**重要なパターン:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## ステップ5: ルーティングの条件関数を定義する

条件付きワークフローと同じ条件関数を使用します。これらは構造化された出力を調べてルーティングを決定します。


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## ステップ6: カスタムディスプレイエグゼキューターを作成する

以前と同じエグゼキューター - 最終的なワークフローの出力を表示します。


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## ステップ7: 環境変数の読み込み

LLMクライアント（GitHub ModelsまたはOpenAI）を設定します。


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## ステップ 8: ミドルウェアを使用してAIエージェントを作成

**重要な違い:** availability_agentを作成する際に、`middleware`パラメータを渡します！

これにより、priority_check_middlewareをエージェントの関数呼び出しパイプラインに注入する方法が実現します。


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ステップ9: ワークフローの構築

以前と同じワークフロー構造 - 利用可能性に基づく条件付きルーティング。


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## ステップ 10: テストケース 1 - パリの通常ユーザー (オーバーライドなし)

通常のユーザーがパリを予約しようとする → 部屋がない → alternative_agentにルートされる


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## ステップ11: テストケース2 - 🌟 パリの優先ユーザー（オーバーライドあり！）

優先メンバーがパリを予約しようとする → 最初は部屋がない → 🌟 ミドルウェアがオーバーライド！ → booking_agentにルートされる

**これがミドルウェアの力を示す重要なデモです！**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## ステップ12: テストケース3 - ストックホルムの優先ユーザー（すでに利用可能）

優先ユーザーがストックホルムを試す → 部屋が利用可能 → 上書きは不要 → booking_agentにルートされる

これは、ミドルウェアが必要な場合にのみ動作することを示しています！


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## 重要なポイントとミドルウェアの概念

### ✅ 学んだこと:

#### **1. 関数ベースのミドルウェアパターン**

ミドルウェアは、シンプルな非同期関数を使用して関数呼び出しを傍受します:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. コンテキストアクセスと結果の上書き**

- `context.function` - 呼び出される関数にアクセス
- `context.arguments` - 関数の引数を読み取る
- `context.kwargs` - 追加のパラメータにアクセス
- `await next(context)` - 関数を実行
- `context.result` - 関数の出力を読み取り/変更

#### **3. ビジネスロジックの実装**

ミドルウェアは優先会員の特典を実装します:
- **通常ユーザー**: 変更なし、標準的なワークフロー
- **優先ユーザー**: 「空きなし」を「空きあり」に上書き
- **条件付きロジック**: 必要な場合のみ上書き

#### **4. 同じワークフロー、異なる結果**

ミドルウェアの力:
- ✅ ワークフロー構造の変更なし
- ✅ ツール関数の変更なし
- ✅ 条件付きルーティングロジックの変更なし
- ✅ ミドルウェアだけ → 異なる動作！

### 🚀 実世界での応用:

1. **VIP/プレミアム機能**
   - プレミアムユーザーのレート制限を上書き
   - リソースへの優先アクセスを提供
   - プレミアム機能を動的に解放

2. **A/Bテスト**
   - ユーザーを異なる実装にルーティング
   - 特定のユーザーで新機能をテスト
   - 段階的な機能展開

3. **セキュリティとコンプライアンス**
   - 関数呼び出しを監査
   - 敏感な操作をブロック
   - ビジネスルールを強制

4. **パフォーマンス最適化**
   - 特定のユーザーの結果をキャッシュ
   - 高コストな操作を可能な限りスキップ
   - 動的なリソース割り当て

5. **エラーハンドリングとリトライ**
   - エラーを優雅にキャッチして処理
   - リトライロジックを実装
   - 代替の実装にフォールバック

6. **ログとモニタリング**
   - 関数の実行時間を追跡
   - パラメータと結果をログ
   - 使用パターンをモニタリング

### 🔑 デコレーターとの主な違い:

| 特徴 | デコレーター | ミドルウェア |
|------|-------------|--------------|
| **スコープ** | 単一の関数 | エージェント内のすべての関数 |
| **柔軟性** | 定義時に固定 | 実行時に動的 |
| **コンテキスト** | 制限あり | エージェント全体のコンテキスト |
| **構成** | 複数のデコレーター | ミドルウェアパイプライン |
| **エージェント対応** | いいえ | はい (エージェント状態へのアクセス) |

### 📚 ミドルウェアを使用するタイミング:

✅ **ミドルウェアを使用する場合:**
- ユーザー/セッション状態に基づいて動作を変更する必要がある
- 複数の関数にロジックを適用したい
- エージェントレベルのコンテキストにアクセスする必要がある
- 横断的な関心事 (ログ、認証など) を実装している

❌ **ミドルウェアを使用しない場合:**
- シンプルな入力検証 (Pydanticを使用)
- 関数固有のロジック (関数内に保持)
- 一度きりの変更 (関数を直接変更)

### 🎓 高度なパターン:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 関連する概念:

- **エージェントミドルウェア**: agent.run() 呼び出しを傍受
- **関数ミドルウェア**: ツール関数呼び出しを傍受 (今回使用したもの!)
- **ミドルウェアパイプライン**: 順番に実行されるミドルウェアのチェーン
- **コンテキスト伝播**: ミドルウェアチェーンを通じて状態を渡す



---

**免責事項**:  
この文書は、AI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を期しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。原文（元の言語で記載された文書）を公式な情報源としてご参照ください。重要な情報については、専門の人間による翻訳をお勧めします。本翻訳の使用に起因する誤解や誤認について、当方は一切の責任を負いかねます。
